In [ ]:
import pandas as pd

Leamos los datos (pero esta vez,  entreguemos los *types* en el comando)

In [ ]:
dataTrips = pd.read_csv("trips_new.csv", dtype={'id': 'int64', 'duration': 'float64', 'start_date': 'object', 'start_station': 'category', 'end_date': 'object', 'end_station': 'category', 'bike_nr': 'category', 'subscription_type': 'category', 'zip_code': 'category', 'birth_date': 'float64', 'gender': 'category'})


In [ ]:
dataTrips['start_date']=dataTrips['start_date'].astype('datetime64[ns]')
dataTrips['end_date']=dataTrips['end_date'].astype('datetime64[ns]')

Terminamos de ajustar los datos: duración en minutos,  sin errores en las estaciones, y viajes menores a 24 horas.

In [ ]:
dataTrips['duration']=dataTrips['duration']/60

In [ ]:
finalData=dataTrips.dropna(subset=['start_station', 'end_station'])
finalData = finalData[finalData['duration']<24*60]

In [ ]:
finalData

## Repaso

Lo que vimos la clase pasada es el paradigma de *split-apply-combine*:
        
![split-apply-combine](split-apply-combine.png)


Esto lo hacemos con el comando `<dataframe>.groupby(<forma_agrupar>).apply(<funciona_a_aplicar>)`

In [ ]:
finalData.groupby('gender')['duration'].mean()

In [ ]:
finalData.groupby(finalData['birth_date'] > 1980)['duration'].max()

In [ ]:
def propCasual(x):
    numCasual = sum(x=="Casual")
    return numCasual/len(x)
finalData.groupby(['start_station'])['subscription_type'].apply(propCasual)

# Ejercicio:
Hubway cobra basado en la duración del viaje. Usuarios *Casual* pagan `$0` si el viaje dura menos de 30 minutos, `$2` si dura entre 30 y 60 minutos, y `$6` para viajes entre 60 a 90 minutos Por cada 30 minutos adicioanles, se cobran `$8` adicionales, con un máximo de `$100`.   Un usuario *Registered* para el 75\% de la tarifa solamente (con un máximo de `$75`, por lo tanto). 

**Ejercicio**: Agregue una columna `fee` que indique cuanto se cobró en cada viaje, y calcule fueron los ingresos por cada tipo de usuario, y también calcule los ingresos de cada año.

## Tablas Pivotes
Una herramienta muy común en algunos softwares de datos (incluyendo Excel) son las tablas pivotes.  Pandas incluye un propio comando para esto.

Agreguemos un par de columnas, como el viaje de cada mes o el dia de la semana

In [ ]:
finalData['month']=finalData['start_date'].dt.month
finalData['day']=finalData['start_date'].dt.day_name()

Podemos agrupar por mas de una columna, por ejemplo, la duración promedio de los viajes por cada dia de la semana los distintos meses.


In [ ]:
finalData.groupby(['month','day'])['duration'].mean()

Un forma mas efectiva de visualizar esto es en una tabla, esto se llama una tabla pivote.  Donde decido que son las fila `index`, las columnas `columns` y los valores a desplegar `values` así como el *agregador* a aplicar a cada dato `aggfunc`. 

In [ ]:
finalData.pivot_table(values='duration', columns='month', index='day', aggfunc='mean')

In [ ]:
finalData.pivot_table(values=['duration','birth_date'], columns='subscription_type', index='month', aggfunc=['mean', 'count'])

# Uniendo Dataframe

 Supongamos que queremos calcular la distancia de cada viaje. Podemos estimarlo a partir de la ubicación (lat/lon) de cada estación, que está en otra tabla de datos station. ¿Como podemos unir estas tablas para usar esos datos?



In [ ]:
dataStations = pd.read_csv("stations.csv", dtype={'id': 'category', 'name': 'string', 'lat': 'float64', 'lon': 'float64'})
dataStations


Para unir datos entre dos o mas DataFrames, se requiere un identificador comun entre las dos, y lo que se genera es una nueva tabla con el "producto cruz" de estas dos tablas.

In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})

In [ ]:
df2

In [ ]:
pd.merge(df1, df2)



En este caso, se definió el identificador común (columna *employee*) y se cruzaron los datos para generar una nueva tabla.

**OJO**: Los datos se cruzan, es decir, si hay mas de una columna que coincide, se pondrán todos. Por ejemplo, supongamos que 'Lisa' también trabaja en 'HR'

In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue', 'Lisa'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
pd.merge(df1, df2)

También puede haber datos faltantes, en cuyo caso se entrega la intersección.

In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue', 'Ann'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue', 'Mike'],
                    'hire_date': [2004, 2008, 2012, 2014, 2003]})
pd.merge(df1, df2)




![merge options](merge.png)

In [ ]:
pd.merge(df1, df2, how='outer')

Vamos a usar el DF de trips para a cada registro agregarle la lat/lng, usando el dato de station recordemos que en trips, esta información está en `start_station` pero en stations, la columna que corresponde es `id`.

In [ ]:
dataStations

In [ ]:
pd.merge(finalData,dataStations,left_on='start_station', right_on='id')

lo que hizo fue "pegar" las columnas correspondientes a cada station como columnas adicionales de trip, por lo que podemos usar las columnas lat y lng de la estación de inicio.  Hagamoslo nuevamente con la estación de termino.

In [ ]:
tmp1=pd.merge(finalData,dataStations,left_on='start_station', right_on='id')
tmp2=pd.merge(tmp1,dataStations,left_on='end_station', right_on='id')
    

Calculemos la distancia entre dos puntos.

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def computeDistance(lat_x, lon_x, lat_y, lon_y):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat_x)
    lon1 = radians(lon_x)
    lat2 = radians(lat_y)
    lon2 = radians(lon_y)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


In [ ]:
computeDistance(52.2296756,21.0122287,52.406374, 16.9251681)

In [ ]:
tmp2['distance']=tmp2.apply(lambda x: computeDistance(x['lat_x'],x['lng_x'],x['lat_y'],x['lng_y']), axis=1)
tmp2